# Insert Overwrite
1. Replace all the data in a table
2. Replace all the data from a specific partition
3. How to handle schema changes

## 1. Replace all the data in a table

In [0]:
%sql
DROP TABLE IF EXISTS demo.delta_lake.gold_companies;

CREATE TABLE demo.delta_lake.gold_companies
(
  company_name STRING,
  founded_date DATE,
  country STRING
);

INSERT INTO demo.delta_lake.gold_companies
(company_name, founded_date, country)
VALUES 
  ("Apple", "1976-04-01", "USA"),
  ("Tencent", "1998-02-01", "China");

SELECT * FROM demo.delta_lake.gold_companies;

In [0]:
%sql
DROP TABLE IF EXISTS demo.delta_lake.bronze_companies;

CREATE TABLE demo.delta_lake.bronze_companies
(
  company_name STRING,
  founded_date DATE,
  country STRING
);

INSERT INTO demo.delta_lake.bronze_companies
(company_name, founded_date, country)
VALUES 
  ("Apple", "1976-04-01", "USA"),
  ("Tencent", "1998-02-01", "China"),
  ("Microsoft", "1975-04-01", "USA"),
  ("Alibaba", "1999-02-01", "China"),
  ("Google", "1998-03-01", "USA")
;


SELECT * FROM demo.delta_lake.bronze_companies;

In [0]:
%sql
-- read data from bronze table and insert overwrite into the gold table
INSERT OVERWRITE demo.delta_lake.gold_companies
  SELECT * 
  FROM demo.delta_lake.bronze_companies;

In [0]:
%sql
select * from demo.delta_lake.gold_companies;

In [0]:
%sql 
desc history demo.delta_lake.gold_companies;

## 2. Replace all the data from a specific partition

In [0]:
%sql 
-- turn gold table into a table with partition
DROP TABLE IF EXISTS demo.delta_lake.gold_companies_partitioned;


CREATE TABLE demo.delta_lake.gold_companies_partitioned
(
  company_name STRING,
  founded_date DATE,
  country STRING
)
PARTITIONED BY (country);


INSERT INTO demo.delta_lake.gold_companies_partitioned
(company_name, founded_date, country)
VALUES 
  ("Apple", "1976-04-01", "USA"),
  ("Tencent", "1998-02-01", "China");


SELECT * FROM demo.delta_lake.gold_companies_partitioned;

In [0]:
%sql 
desc extended demo.delta_lake.gold_companies_partitioned;

In [0]:
%sql
-- create bronze table containing new data for usa companies
DROP TABLE IF EXISTS demo.delta_lake.bronze_companies_usa;

CREATE TABLE demo.delta_lake.bronze_companies_usa
(
  company_name STRING,
  founded_date DATE,
  country STRING
);

INSERT INTO demo.delta_lake.bronze_companies_usa
(company_name, founded_date, country)
VALUES 
  ("Microsoft", "1975-04-01", "USA"),
  ("Google", "1998-03-01", "USA"),
  ("Facebook", "2004-02-01", "USA");
    
SELECT * FROM demo.delta_lake.bronze_companies_usa;


In [0]:
%sql
-- overwrite USA partition
INSERT OVERWRITE demo.delta_lake.gold_companies_partitioned 
PARTITION (country = 'USA')
SELECT
  company_name,
  founded_date
FROM demo.delta_lake.bronze_companies_usa;

In [0]:
%sql
select * from demo.delta_lake.gold_companies_partitioned;

## 3. How to handle schema changes
Insert Overwrite -> use to overwrite the data in a table or a partition when there are no schema changes.

Create or replace table -> use when there are schema changes